In [1]:
import sys
import asyncio
from dotenv import load_dotenv

load_dotenv()

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__

## Local MCP server

In [2]:
import sys
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": sys.executable,
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [9]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [4]:
prompt

'\n    You are a helpful assistant that answers user questions about LangChain, LangGraph and LangSmith.\n\n    You can use the following tools/resources to answer user questions:\n    - search_web: Search the web for information\n    - github_file: Access the langchain-ai repo files\n\n    If the user asks a question that is not related to LangChain, LangGraph or LangSmith, you should say "I\'m sorry, I can only answer questions about LangChain, LangGraph and LangSmith."\n\n    You may try multiple tool and resource calls to answer the user\'s question.\n\n    You may also ask clarifying questions to the user to better understand their question.\n    '

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='e3116f0c-16ca-479a-83a5-5560f85dbdb2'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 271, 'total_tokens': 363, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpTiwkMZhMcnajrgO4hlRakaBYhHq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b44c0-8252-7370-b185-d29636f95a55-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters'}, 'id': 'call_MWq9JkVis493S1n9qhtIvvKN', 'type': 'tool_call'}], usage_metadata={'input_toke

## Online MCP

In [12]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/Denver"
            ]
        }
    }
)

tools = await client.get_tools()

In [13]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
)

In [14]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='7d44efa7-b569-4379-acc5-cc2364a8d285'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 282, 'prompt_tokens': 293, 'total_tokens': 575, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpTlCbNy2Vf1Sc6XS424pgRzLKHaW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b44c2-a105-7813-9bfe-a34fe1eb6fc9-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/Denver'}, 'id': 'call_uuOWp5bV1UC1IFST5arvXcaf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 293, 'output_tokens': 28